In [15]:

import os
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

In [16]:
# Image Denoising Performance
path = os.path.join('denoised_outputs')
dfs = []
for model in os.listdir(path):
    model_path = os.path.join(path, model)
    for noise_level in os.listdir(model_path):
        denoised_results = pd.read_json(os.path.join(model_path, noise_level, 'denoised_results.json'))
        denoised_results['model'] = model
        denoised_results['noise_level'] = noise_level
        dfs.append(denoised_results)
        
denoised_df = pd.concat(dfs).reset_index().drop(columns=['index'])
denoised_df

,test_loss,test_psnr,test_ssim,model,noise_level
0,0.029225,15.975712,0.598784,vae,0.1
1,0.032382,15.373611,0.593488,vae,0.5
2,0.033620,14.737119,0.585927,vae,0.7
3,0.002716,26.024093,0.773958,vae_with_custom_loss,0.1
4,0.002710,26.037799,0.767546,vae_with_custom_loss,0.5
5,0.003844,24.420916,0.745324,vae_with_custom_loss,0.7
6,0.000644,31.912840,0.861270,vae_with_skip_connections,0.1
7,0.002090,26.805023,0.807092,vae_with_skip_connections,0.5
8,0.004260,23.708087,0.752398,vae_with_skip_connections,0.7
9,0.000624,32.049979,0.864234,vae_with_skip_connections_and_custom_loss,0.1


In [17]:
# Uncertainty Quantification (MC Dropout)
path = os.path.join('mc_dropout', 'model_outputs')
dfs = []
for config in os.listdir(path):
    mc_dropout_df = pd.read_json(os.path.join(path, config, 'mc_dropout_results.json'))
    model, noise_level = config.split('_noise_std_')
    mc_dropout_df = mc_dropout_df.mean().to_frame().T
    mc_dropout_df['model'] = model
    mc_dropout_df['noise_level'] = noise_level
    mc_dropout_df = mc_dropout_df.drop(['batch_size', 'test_loss', 'test_psnr', 'test_ssim'], axis=1)
    mc_dropout_df = mc_dropout_df.rename(columns={
        'max_uncertainty': 'max_uncertainty_mc_dropout',
        'avg_uncertainty': 'avg_uncertainty_mc_dropout'
    })
    dfs.append(mc_dropout_df)
    
mc_dropout_df = pd.concat(dfs).reset_index().drop(columns=['index'])
mc_dropout_df

,avg_uncertainty_mc_dropout,max_uncertainty_mc_dropout,model,noise_level
0,3.318175e-03,0.061221,vae,0.1
1,3.088913e-03,0.066108,vae,0.5
2,2.659669e-04,0.008513,vae,0.7
3,7.694145e-04,0.156983,vae_with_custom_loss,0.1
4,7.587765e-04,0.131829,vae_with_custom_loss,0.5
5,7.991731e-04,0.127682,vae_with_custom_loss,0.7
6,7.036625e-07,0.001482,vae_with_skip_connections_and_custom_loss,0.1
7,1.247225e-05,0.004512,vae_with_skip_connections_and_custom_loss,0.5
8,3.597750e-05,0.003562,vae_with_skip_connections_and_custom_loss,0.7
9,3.828000e-07,0.000046,vae_with_skip_connections,0.1


In [18]:
# Uncertainty Quantification (Deep Ensemble)
path = os.path.join('deep_ensembles', 'ensemble_model_outputs')
dfs = []
for config in os.listdir(path):
    deep_ensembles_df = pd.read_json(os.path.join(path, config, 'results.json'))
    model, noise_level = config.split('_noise_std_')
    deep_ensembles_df = deep_ensembles_df.mean().to_frame().T
    deep_ensembles_df['model'] = model
    deep_ensembles_df['noise_level'] = noise_level
    deep_ensembles_df = deep_ensembles_df.rename(columns={
        'epistemic': 'mean_epistemic_uncertainty_deep_ensemble',
        'aleatoric': 'mean_aleatoric_uncertainty_deep_ensemble',
        'total': 'mean_total_uncertainty_deep_ensemble',
    })
    dfs.append(deep_ensembles_df)

deep_ensembles_df = pd.concat(dfs).reset_index().drop(columns=['index'])
deep_ensembles_df

,mean_epistemic_uncertainty_deep_ensemble,mean_aleatoric_uncertainty_deep_ensemble,mean_total_uncertainty_deep_ensemble,model,noise_level
0,0.000742,2.728510e-03,0.003471,vae,0.1
1,0.001407,2.805630e-03,0.004213,vae,0.5
2,0.001095,2.997092e-03,0.004092,vae,0.7
3,0.000435,4.413275e-05,0.000479,vae_with_custom_loss,0.1
4,0.000455,4.970669e-05,0.000504,vae_with_custom_loss,0.5
5,0.000445,4.006750e-05,0.000485,vae_with_custom_loss,0.7
6,0.000059,6.299938e-07,0.000060,vae_with_skip_connections_and_custom_loss,0.1
7,0.000213,1.932426e-05,0.000232,vae_with_skip_connections_and_custom_loss,0.5
8,0.000847,4.074904e-05,0.000888,vae_with_skip_connections_and_custom_loss,0.7
9,0.000057,4.303752e-07,0.000057,vae_with_skip_connections,0.1


In [19]:
# Combine all DFs
df = denoised_df.merge(mc_dropout_df, on=['model', 'noise_level'], how='inner')
df = df.merge(deep_ensembles_df, on=['model', 'noise_level'], how='inner')
# df = df.round(4)
df

,test_loss,test_psnr,test_ssim,model,noise_level,avg_uncertainty_mc_dropout,max_uncertainty_mc_dropout,mean_epistemic_uncertainty_deep_ensemble,mean_aleatoric_uncertainty_deep_ensemble,mean_total_uncertainty_deep_ensemble
0,0.029225,15.975712,0.598784,vae,0.1,3.318175e-03,0.061221,0.000742,2.728510e-03,0.003471
1,0.032382,15.373611,0.593488,vae,0.5,3.088913e-03,0.066108,0.001407,2.805630e-03,0.004213
2,0.033620,14.737119,0.585927,vae,0.7,2.659669e-04,0.008513,0.001095,2.997092e-03,0.004092
3,0.002716,26.024093,0.773958,vae_with_custom_loss,0.1,7.694145e-04,0.156983,0.000435,4.413275e-05,0.000479
4,0.002710,26.037799,0.767546,vae_with_custom_loss,0.5,7.587765e-04,0.131829,0.000455,4.970669e-05,0.000504
5,0.003844,24.420916,0.745324,vae_with_custom_loss,0.7,7.991731e-04,0.127682,0.000445,4.006750e-05,0.000485
6,0.000644,31.912840,0.861270,vae_with_skip_connections,0.1,3.828000e-07,0.000046,0.000057,4.303752e-07,0.000057
7,0.002090,26.805023,0.807092,vae_with_skip_connections,0.5,3.214896e-06,0.001261,0.000580,6.093813e-06,0.000586
8,0.004260,23.708087,0.752398,vae_with_skip_connections,0.7,2.465368e-05,0.002309,0.001571,2.303337e-05,0.001594
9,0.000624,32.049979,0.864234,vae_with_skip_connections_and_custom_loss,0.1,7.036625e-07,0.001482,0.000059,6.299938e-07,0.000060


In [20]:
# Image Denoising Performance
path = os.path.join('classification', 'model_outputs')
dfs = []
for dataset_type in os.listdir(path):
    dataset_path = os.path.join(path, dataset_type)
    if dataset_type == 'original':
        df = pd.read_json(os.path.join(dataset_path, 'eval.json'))
        df['dataset_type'] = dataset_type
        df['noise_level'] = 0
        dfs.append(df)
    else:
        for noise_level in os.listdir(dataset_path):
            df = pd.read_json(os.path.join(dataset_path, noise_level, 'eval.json'))
            df['dataset_type'] = dataset_type
            df['noise_level'] = noise_level
            dfs.append(df)
        
classification_df = pd.concat(dfs).reset_index().drop(columns=['index'])
classification_df[(classification_df['mode'] == 'test') & (classification_df['epoch'] == 30)]

,epoch,mode,accuracy,loss,dataset_type,noise_level
59,30,test,24.833333,43.563199,original,0
119,30,test,13.416667,45.986756,vae,0.1
179,30,test,11.166667,46.544644,vae,0.5
239,30,test,7.500000,47.219019,vae,0.7
299,30,test,20.500000,46.200661,vae_with_custom_loss,0.1
359,30,test,21.666667,44.512192,vae_with_custom_loss,0.5
419,30,test,22.916667,44.879175,vae_with_custom_loss,0.7
479,30,test,18.583333,50.714146,vae_with_skip_connections,0.1
539,30,test,21.916667,43.521597,vae_with_skip_connections,0.5
599,30,test,20.166667,43.911597,vae_with_skip_connections,0.7


In [ ]:
name_map = {
    'vae': 'VAE',
    'vae_with_custom_loss': 'VAE + CL',
    'vae_with_skip_connections': 'VAE + SC',
    'vae_with_skip_connections_and_custom_loss': 'VAE + SC + CL',
}

RAW_LABELS = {
    1: 'Atelectasis',
    2: 'Cardiomegaly',
    3: 'Effusion',
    4: 'Inflitration',
    5: 'Mass',
    6: 'Nodule',
    7: 'Pneumonia',
    8: 'Pneumothorax',
    9: 'Consolidation',
    10: 'Edema',
    11: 'Emphysema',
    12: 'Fibrosis'
}

# Configuration
base_path = 'denoised_outputs'
class_indices = [i for i in range(2)]  # Two sample images
noise_level = '0.1'
model_names = list(name_map.keys())
col_titles = ['Original', f'Noisy (σ={noise_level})'] + model_names

fig, axes = plt.subplots(len(class_indices), len(col_titles), figsize=(10, 3))
fig.subplots_adjust(wspace=0.05, hspace=0.1)

for row, class_idx in enumerate(class_indices):
    for col, title in enumerate(col_titles):
        ax = axes[row, col]
        try:
            if title == 'Original':
                img_dir = os.path.join(base_path, model_names[0], noise_level, 'parsed_images', 'original', str(class_idx))
            elif 'Noisy' in title:
                img_dir = os.path.join(base_path, model_names[0], noise_level, 'parsed_images', 'noisy', str(class_idx))
            else:
                img_dir = os.path.join(base_path, title, noise_level, 'parsed_images', 'denoised', str(class_idx))
            
            img_file = os.listdir(img_dir)[0]
            img = Image.open(os.path.join(img_dir, img_file))
            ax.imshow(img, cmap='gray')
        except Exception as e:
            print(f"Missing or error loading image for class {class_idx}, column '{title}': {e}")
            ax.axis('off')
            continue
        
        # Make sure grid is off
        ax.grid(False)
        # Remove all axes, spines, ticks, etc.
        ax.set_xticks([])
        ax.set_yticks([])
        for spine in ax.spines.values():
            spine.set_visible(False)
        # ax.axis('off')
        
        # Column titles (top row only)
        if row == 0:
            ax.set_title(name_map.get(title, title), fontsize=10)
        
        # Set row label only for first column
        if col == 0:
            ax.set_ylabel(f"{RAW_LABELS[class_idx + 1]}", fontsize=12, rotation=90, labelpad=10, va='center')

# Save figure
plt.savefig("neurips_denoising_comparison.png", bbox_inches='tight', dpi=300)
plt.close()

In [29]:
name_map = {
    'vae': 'VAE',
    'vae_with_custom_loss': 'VAE + CL',
    'vae_with_skip_connections': 'VAE + SC',
    'vae_with_skip_connections_and_custom_loss': 'VAE + SC + CL',
}

RAW_LABELS = {
    1: 'Atelectasis',
    2: 'Cardiomegaly',
    3: 'Effusion',
    4: 'Inflitration',
    5: 'Mass',
    6: 'Nodule',
    7: 'Pneumonia',
    8: 'Pneumothorax',
    9: 'Consolidation',
    10: 'Edema',
    11: 'Emphysema',
    12: 'Fibrosis'
}

# Configuration
base_path = 'denoised_outputs'
class_indices = [i for i in range(5)]  # Two sample images
noise_level = '0.1'
model_names = list(name_map.keys())
col_titles = ['Original', f'Noisy (σ={noise_level})'] + model_names

fig, axes = plt.subplots(len(col_titles), len(class_indices), figsize=(10, 5))
fig.subplots_adjust(wspace=0.05, hspace=0.1)

for row, title in enumerate(col_titles):
    for col, class_idx in enumerate(class_indices):
        ax = axes[row, col]
        try:
            if title == 'Original':
                img_dir = os.path.join(base_path, model_names[0], noise_level, 'parsed_images', 'original', str(class_idx))
            elif 'Noisy' in title:
                img_dir = os.path.join(base_path, model_names[0], noise_level, 'parsed_images', 'noisy', str(class_idx))
            else:
                img_dir = os.path.join(base_path, title, noise_level, 'parsed_images', 'denoised', str(class_idx))

            img_file = os.listdir(img_dir)[0]
            img = Image.open(os.path.join(img_dir, img_file))
            ax.imshow(img, cmap='gray')
        except Exception as e:
            print(f"Missing or error loading image for class {class_idx}, row '{title}': {e}")
            ax.axis('off')
            continue

        ax.grid(False)
        ax.set_xticks([])
        ax.set_yticks([])
        for spine in ax.spines.values():
            spine.set_visible(False)

        # Add row titles (model names) on the leftmost column
        if col == 0:
            ax.set_ylabel(name_map.get(title, title), fontsize=10, rotation=0, labelpad=5, va='center', ha='right')

        # Add column headers (class labels) only on the top row
        if row == 0:
            ax.set_title(RAW_LABELS[class_idx + 1], fontsize=10)

plt.savefig("neurips_denoising_comparison_horizontal.png", bbox_inches='tight', dpi=300)
plt.close()
